In [1]:
import pandas as pd
import numpy as np

In [2]:
df_migration = pd.read_csv("../Data/Number_of_Migants_to_Muncipalties_per_year.csv")

df_housing_prices = pd.read_csv("../Data_processing/Cleaned_Data_Sets/2005_2024_with_municipalities/All_prefectures_buildings.csv")

In [9]:
df_migration.columns

Index(['Destination', 'area_code', 'value', 'Year'], dtype='object')

In [8]:
df_housing_prices.columns

Index(['Type', 'City,Town,Ward,Village code', 'Prefecture', 'Location',
       'TotalTransactionValue', 'Area', 'Frontage', 'TotalFloorArea',
       'ConstructionYear', 'BuildingCoverageRatio', 'FloorAreaRatio',
       'Quarter', 'Year', 'RegionCommercialArea', 'RegionIndustrialArea',
       'RegionPotentialResidentialArea', 'RegionResidentialArea',
       'MunicipalityCategory', 'Region_Chubu', 'AverageTimeToStation',
       'FloorAreaGreaterFLag', 'BeforeWarFlag', 'frontage_greater_than_50',
       'AreaGreaterFlag', 'Region_Chugoku', 'Region_Hokkaido', 'Region_Kansai',
       'Region_Kanto', 'Region_Kyushu', 'Region_Shikoku', 'Region_Tohoku'],
      dtype='object')

In [10]:
# First, rename columns to align for merging
df_migration_renamed = df_migration.rename(columns={
    'area_code': 'City,Town,Ward,Village code',
    'value': 'Migration',
    'Year': 'MigrationYear'
})

# Create a 'TargetYear' in housing_prices for joining
df_housing_prices['TargetYear'] = df_housing_prices['Year'] - 1 #assumption is that migration has a 1 year lag on housing prices

# Attempt the direct merge on area_code and target year
merged = pd.merge(
    df_housing_prices,
    df_migration_renamed,
    how='left',
    left_on=['City,Town,Ward,Village code', 'TargetYear'],
    right_on=['City,Town,Ward,Village code', 'MigrationYear']
)

# Now fill missing Migration values using the earliest year per area_code
# First, get the earliest year migration value per area_code
earliest_migration = (
    df_migration_renamed.sort_values('MigrationYear')
    .drop_duplicates('City,Town,Ward,Village code')
    [['City,Town,Ward,Village code', 'Migration']]
)

# Fill in missing Migration values in merged dataframe
merged = pd.merge(
    merged,
    earliest_migration,
    how='left',
    on='City,Town,Ward,Village code',
    suffixes=('', '_earliest')
)

# Use the found migration value or fall back to earliest
merged['Migration'] = merged['Migration'].combine_first(merged['Migration_earliest'])

# Drop helper columns
merged = merged.drop(columns=['Migration_earliest', 'TargetYear', 'MigrationYear'])


In [12]:
df_migration[df_migration['area_code'] == 20201]

,Destination,area_code,value,Year
22334,Nagano-shi,20201,23585,2008
22335,Nagano-shi,20201,23073,2009
22336,Nagano-shi,20201,22157,2010
22337,Nagano-shi,20201,22502,2011
22338,Nagano-shi,20201,21826,2012
22339,Nagano-shi,20201,22131,2013
22340,Nagano-shi,20201,21156,2014
22341,Nagano-shi,20201,22877,2015
22342,Nagano-shi,20201,21297,2016
22343,Nagano-shi,20201,21628,2017


In [11]:
merged.head()


,Type,"City,Town,Ward,Village code",Prefecture,Location,TotalTransactionValue,Area,Frontage,TotalFloorArea,ConstructionYear,BuildingCoverageRatio,...,AreaGreaterFlag,Region_Chugoku,Region_Hokkaido,Region_Kansai,Region_Kanto,Region_Kyushu,Region_Shikoku,Region_Tohoku,Destination,Migration
0,Residential Land(Land and Building),20201,Nagano Prefecture,Nagano City,27000000,175,9.0,100,2024,60.0,...,False,False,False,False,False,False,False,False,Nagano-shi,20101.0
1,Residential Land(Land and Building),20201,Nagano Prefecture,Nagano City,27000000,125,9.0,100,2019,60.0,...,False,False,False,False,False,False,False,False,Nagano-shi,20376.0
2,Residential Land(Land and Building),20201,Nagano Prefecture,Nagano City,6500000,90,10.0,95,1996,60.0,...,False,False,False,False,False,False,False,False,Nagano-shi,20587.0
3,Residential Land(Land and Building),20201,Nagano Prefecture,Nagano City,27000000,135,8.5,80,2021,60.0,...,False,False,False,False,False,False,False,False,Nagano-shi,20587.0
4,Residential Land(Land and Building),20201,Nagano Prefecture,Nagano City,67000000,340,4.0,200,2020,60.0,...,False,False,False,False,False,False,False,False,Nagano-shi,22103.0


In [18]:
merged.to_csv("/home/brianmcgloughlin/Japanese_future_real_estate_prices/Data_processing/Cleaned_Data_Sets/All_prefectures_buildings_with_migration.csv", index=False)